# League of Legend(LoL) Data about Rank Game Result of Korea from bronze to diamond
#### Data : https://www.kaggle.com/datasets/park123/lol-data

## 데이터
### match_15m
> 15분까지의 게임 결과

### match_full_time
> 전체 게임 결과

### 라이브러리 설치

In [1]:
import pandas as pd
import os
import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

### 데이터 불러오기

In [2]:
PATH_1 = 'lol_data/'
PATH_2 = 'lol_data/champ/'

file_list_1 = os.listdir(PATH_1)
file_list_2 = os.listdir(PATH_2)
csv_list_1 = list()
csv_list_2 = list()

for file in file_list_1:
    if file.split(".")[-1] == 'csv':
        csv_list_1.append(file)

for file in file_list_2:
    if file.split(".")[-1] == 'csv':
        csv_list_2.append(file)

In [3]:
game = []
champ = []

for j in csv_list_1:
    game.append(j.split('.csv')[0])
    
for i in csv_list_2:
    champ.append(i.split('.csv')[0])

In [4]:
for name, file in zip(game, csv_list_1):
    globals()[name] = pd.read_csv(PATH_1 + file, encoding = 'cp949')
    globals()[name] = globals()[name].set_index('matchId')

for name, file in zip(champ, csv_list_2):
    globals()[name] = pd.read_csv(PATH_2 + file, encoding = 'cp949')
    globals()[name] = globals()[name].set_index('matchId')

### 데이터 살펴보기

In [5]:
match_15m.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 4569533952 to 4578342300
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   blue_kill       5000 non-null   int64 
 1   red_kill        5000 non-null   int64 
 2   blue_gold       5000 non-null   int64 
 3   red_gold        5000 non-null   int64 
 4   blue_tower      5000 non-null   int64 
 5   blue_inhibitor  5000 non-null   int64 
 6   red_tower       5000 non-null   int64 
 7   red_inhibitor   5000 non-null   int64 
 8   blue_monster    5000 non-null   object
 9   red_monster     5000 non-null   object
 10  current_time    5000 non-null   int64 
 11  blue_win        5000 non-null   int64 
dtypes: int64(10), object(2)
memory usage: 507.8+ KB


In [6]:
match_full_time.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 4569533952 to 4578342300
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   makeTime              5000 non-null   object
 1   tier                  5000 non-null   object
 2   blue_kill             5000 non-null   int64 
 3   red_kill              5000 non-null   int64 
 4   blue_gold             5000 non-null   int64 
 5   red_gold              5000 non-null   int64 
 6   blue_tower            5000 non-null   int64 
 7   blue_inhibitor        5000 non-null   int64 
 8   red_tower             5000 non-null   int64 
 9   red_inhibitor         5000 non-null   int64 
 10  blue_monster          5000 non-null   object
 11  red_monster           5000 non-null   object
 12  blue_firstBlood       5000 non-null   bool  
 13  blue_firstTower       5000 non-null   bool  
 14  blue_firstInhibitor   5000 non-null   bool  
 15  blue_firstBaron       5000 n

In [7]:
Aatrox.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305 entries, 4620465190 to 4606817633
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   gameDuration  305 non-null    int64  
 1   goldRatio     305 non-null    float64
 2   killRatio     305 non-null    float64
 3   deathRatio    305 non-null    float64
 4   damageRatio   305 non-null    float64
 5   tier          305 non-null    object 
 6   lane          305 non-null    object 
 7   blue_team     305 non-null    bool   
 8   win           305 non-null    bool   
dtypes: bool(2), float64(4), int64(1), object(2)
memory usage: 19.7+ KB


## 1. 티어별로 진영에 따라 승률이 다를까?
### 게임 수는 티어별로 각각 1000개

#### 필요한 컬러만 뽑아오기

In [8]:
blue_win = match_full_time[['tier', 'blue_win']]

#### 티어별로 블루팀 승리 수 구하기

In [9]:
tier = blue_win.groupby('tier').sum()

#### 인덱스 재정렬

In [10]:
tier = tier.reindex(index = ['BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND'])

#### 시각화

In [11]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=tier.index, 
        y=tier['blue_win'],
        text=tier['blue_win'], 
        textposition='auto',
        textfont={'size' : 30},
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of Blue team wins by tier</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "Tier from BRONZE to DIAMOND",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The number of Blue team wins"
        },
        "template":'plotly_white'
    }, width=900,height=800
)
fig.show()

## 2. 사람들은 주로 몇시에 게임을 할까?

#### 필요한 컬럼만 뽑아오기

In [12]:
time = match_full_time[['makeTime']]

#### Time 데이터로 바꾸기

In [13]:
time['makeTime'] = pd.to_datetime(time['makeTime'], format='%Y-%m-%d %H:%M', errors='raise')

#### 시간열 생성하기

In [14]:
time['hour'] = time['makeTime'].dt.hour

#### 시간별로 정리

In [15]:
time_hour = time.groupby('hour').count()

#### 시각화

In [16]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=time_hour.index, 
        y=time_hour['makeTime'],
        text=time_hour['makeTime'], 
        textposition='auto',
        textfont={'size' : 10},
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of game by hour</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "hour",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            },
            'dtick': 1
        },
        "yaxis": {
            "title": "The number of game"
        },
        "template":'plotly_white'
    }
)
fig.show()

## 3. 사람들은 어떤 챔피언을 가장 많이 사용할까?

### 데이터 전처리

In [17]:
champions = []
play = []

for i in champ:
    champions.append(i)
    play.append(len(globals()[i]))

In [18]:
champion = pd.DataFrame(zip(champions, play))

In [19]:
champion = champion.rename(columns = {0 : 'champions', 
                                      1 : 'play'})

In [20]:
champion = champion.set_index('champions')

#### 플레이 수가 많은 챔피언 순으로 정렬하기

In [21]:
champion = champion.sort_values('play', ascending = False)

#### 시각화

In [22]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=champion.index, 
        y=champion['play'],
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of play games by champion</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The number of play games"
        },
        "template":'plotly_white'
    }
)
fig.show()

### 한눈에 보기 어려우므로 상위 30개만 확인하기

In [23]:
top_30_champion = champion.head(30)

#### 상위 3개의 색 바꾸기

In [24]:
colors = ['#F29F05',] * len(top_30_champion.index)
colors[0] = '#F54F29'
colors[1] = '#F54F29'
colors[2] = '#F54F29'

In [25]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=top_30_champion.index, 
        y=top_30_champion['play'],
        text=top_30_champion['play'], 
        textposition='auto',
        marker_color = colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The number of play games by champion</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The number of play games"
        },
        "template":'plotly_white'
    }
)
fig.show()

## 4. 첫 바론을 먹으면 게임을 이길 수 있을까?

#### 필요한 컬럼만 뽑아오기

In [26]:
baron = match_full_time[['blue_firstBaron', 'blue_win']]

#### 중복된 값 정리하기

In [27]:
baron = baron.groupby('blue_firstBaron').sum()

#### 인덱스 이름 바꾸기

In [28]:
baron = baron.rename(index = {False : 'X', 
                               True : 'O'})

#### 시각화

In [29]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=baron.index, 
        y=baron['blue_win'],
        marker_color = '#F29F05',
        text=baron['blue_win'], 
        textposition='auto',
        textfont={'size' : 20}
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The relationship between the first Baron and the victory</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "The first Baron",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The number of the victory"
        },
        "template":'plotly_white',
    },
    width=800,height=400
)
fig.show()


### 4-1. 티어별로 나누어서 확인해보기

#### 필요한 컬럼만 뽑아오기

In [30]:
tier_baron = match_full_time[['blue_firstBaron', 'blue_win', 'tier']]

#### 중복된 값 정리하기

In [31]:
tier_baron = tier_baron.groupby(['tier', 'blue_firstBaron']).sum()

#### 인덱스 이름 바꾸기

In [32]:
tier_baron = tier_baron.rename(level = 1,index = {False : 'X', True : 'O'})

#### 멀티인덱스 풀기

In [33]:
tier_baron = tier_baron.reset_index()

#### 시각화를 위해 pivot함수 사용하기

In [34]:
tier_baron = tier_baron.pivot(index = 'tier', columns = 'blue_firstBaron', values = 'blue_win')

#### 인덱스 재정렬

In [35]:
tier_baron = tier_baron.reindex(index = ['BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND'])

#### 시각화
* 참고 : https://2colors.colorion.co/#google_vignette

In [36]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=tier_baron.index,
        y=tier_baron['O'],
        marker_color = '#50586C',
        text=tier_baron['O'], 
        textposition='auto',
        textfont={'size' : 20},
        name = 'O'
    )
)
fig.add_trace(
    go.Bar(
        x=tier_baron.index,
        y=tier_baron['X'],
        marker_color = '#DCE2F0',
        text=tier_baron['X'], 
        textposition='auto',
        textfont={'size' : 20},
        name = 'X'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The relationship between the first Baron and the victory by tier</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "Tier",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The number of the victory"
        },
        "template":'plotly_white',
    }
)
fig.show()


## 5. 어떤 챔피언이 제일 승률이 높을까?

### 데이터 전처리

#### 빈 데이터 프레임 만들기

In [37]:
winning_rate = pd.DataFrame(columns = ['champion', 'winning_rate'])

#### 빈 데이터 프레임에 값 넣기

In [38]:
for i, j in enumerate(champ):
    winning_rate.loc[i] = (j, round((globals()[j]['win'].sum()/len(globals()[j]))*100, 1))

#### 승률이 높은 순으로 정렬하기

In [39]:
winning_rate = winning_rate.sort_values('winning_rate', ascending = False)

#### 챔피언으로 인덱스 설정하기

In [40]:
winning_rate = winning_rate.set_index('champion')

#### 시각화

In [41]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=winning_rate.index,
        y=winning_rate['winning_rate'],
        mode = 'lines+markers+text',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning"
        },
        "template":'plotly_white'
    }
)
fig.show()

### 한눈에 보기 어려우므로 상위 30개만 확인하기

In [42]:
top_30_winning_rate = winning_rate.head(30)

In [43]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_winning_rate.index,
        y=top_30_winning_rate['winning_rate'],
        mode = 'lines+markers+text',
        text = top_30_winning_rate['winning_rate'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()

### 5-1. 티어별 챔피언 승률 확인해보기

#### 데이터 전처리

In [44]:
first_doc = True

for i in champ:
    if first_doc:
        tier_winning_rate, first_doc = round((globals()[i].groupby('tier')['win'].sum()/globals()[i].groupby('tier')['win'].count())*100, 1), False
    else:
        tier_winning_rate = pd.concat([tier_winning_rate, round((globals()[i].groupby('tier')['win'].sum()/globals()[i].groupby('tier')['win'].count())*100, 1)], axis = 1)

#### 컬럼명 변경하기

In [45]:
tier_winning_rate.columns = champ

#### 시각화를 위해 전치

In [46]:
tier_winning_rate = tier_winning_rate.T

#### 컬럼 순서 재설정

In [47]:
tier_winning_rate = tier_winning_rate[['BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']]

#### 티어별로 따로 데이터프레임 만들기

In [48]:
BRONZE_winning_rate = tier_winning_rate[['BRONZE']]
SILVER_winning_rate = tier_winning_rate[['SILVER']]
GOLD_winning_rate = tier_winning_rate[['GOLD']]
PLATINUM_winning_rate = tier_winning_rate[['PLATINUM']]
DIAMOND_winning_rate = tier_winning_rate[['DIAMOND']]

#### 상위 30개만 확인하기

In [49]:
top_30_BRONZE_winning_rate = BRONZE_winning_rate.sort_values('BRONZE', ascending = False).head(30)
top_30_SILVER_winning_rate = SILVER_winning_rate.sort_values('SILVER', ascending = False).head(30)
top_30_GOLD_winning_rate = GOLD_winning_rate.sort_values('GOLD', ascending = False).head(30)
top_30_PLATINUM_winning_rate = PLATINUM_winning_rate.sort_values('PLATINUM', ascending = False).head(30)
top_30_DIAMOND_winning_rate = DIAMOND_winning_rate.sort_values('DIAMOND', ascending = False).head(30)

#### 시각화(브론즈)

In [50]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_BRONZE_winning_rate.index,
        y=top_30_BRONZE_winning_rate['BRONZE'],
        mode = 'lines+markers+text',
        text = top_30_BRONZE_winning_rate['BRONZE'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion on BRONZE</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning on BRONZE"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()

#### 시각화(실버)

In [51]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_SILVER_winning_rate.index,
        y=top_30_SILVER_winning_rate['SILVER'],
        mode = 'lines+markers+text',
        text = top_30_SILVER_winning_rate['SILVER'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion on SILVER</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning on SILVER"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()

#### 시각화(골드)

In [52]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_GOLD_winning_rate.index,
        y=top_30_GOLD_winning_rate['GOLD'],
        mode = 'lines+markers+text',
        text = top_30_GOLD_winning_rate['GOLD'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion on GOLD</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning on GOLD"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()

#### 시각화(플레)

In [53]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_PLATINUM_winning_rate.index,
        y=top_30_PLATINUM_winning_rate['PLATINUM'],
        mode = 'lines+markers+text',
        text = top_30_PLATINUM_winning_rate['PLATINUM'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion on PLATINUM</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning on PLATINUM"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()

#### 시각화(다이아)

In [54]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=top_30_DIAMOND_winning_rate.index,
        y=top_30_DIAMOND_winning_rate['DIAMOND'],
        mode = 'lines+markers+text',
        text = top_30_DIAMOND_winning_rate['DIAMOND'],
        textposition='top center',
        marker_color = '#F29F05'
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>The rate of winning by champion on DIAMOND</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 20
            }
        },
        "xaxis": {
            "title": "champion",
            "showticklabels":True,
            "tickfont": {
                "size": 15                
            }
        },
        "yaxis": {
            "title": "The rate of winning on DIAMOND"
        },
        "template":'plotly_white'
    },
    width=1800,height=600
)
fig.show()